In [50]:
import torch
import numpy as np
import pandas as pd
import os
import statistics
from transformers import BertTokenizer, BertForPreTraining, BertModel
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import plotly
import plotly.express as px

# Utilitary functions

In [52]:
def fix_dataset(dataset):
    
    # Check is all questions in 'question1' and 'question2' are str
    filter = np.array([not isinstance(s1, str) for s1 in dataset['question1'].tolist()]) | np.array([not isinstance(s2, str) for s2 in dataset['question2'].tolist()])
    indexes_to_drop = dataset[filter].index
    
    # drop lines that are not
    if not len(indexes_to_drop):
        print("All rows are corrects")
    else:
        print("Removing the following lines: ")
        print(dataset.loc[indexes_to_drop])
        dataset = dataset.drop(indexes_to_drop)
    
    return dataset

## Import data

In [53]:
# Paths & Variables

data_path = "data/quora-question-pairs"
train_file = "train.csv"
test_pos_file = "test.csv"
label_file = "sample_submission.csv"

# Reading
train = pd.read_csv(os.path.join(data_path, train_file), index_col = 0)
test_pos = pd.read_csv(os.path.join(data_path, test_pos_file), index_col = 0)
y_label = pd.read_csv(os.path.join(data_path, label_file), index_col = 0)

# Fix datasets for NaN values in question1 or question2
train = fix_dataset(train)
test_pos = fix_dataset(test_pos)

# join test and y_label
test_pos = test_pos.join(y_label, on = 'test_id', how = 'left')

# test set contains only positive, labels; suffle to create negative examples
test_neg = test_pos.copy()
test_neg['question1'] = np.random.permutation(test_neg['question1'])
test_neg['is_duplicate'] = 0

# Create final test set
test = pd.concat([test_pos, test_neg], ignore_index = True)

# Reset indexes
train.reset_index(drop = True, inplace = True)
test.reset_index(drop = True, inplace = True)


Removing the following lines: 
          qid1    qid2                         question1  \
id                                                         
105780  174363  174364    How can I develop android app?   
201841  303951  174364  How can I create an Android app?   
363362  493340  493341                               NaN   

                                                question2  is_duplicate  
id                                                                       
105780                                                NaN             0  
201841                                                NaN             0  
363362  My Chinese name is Haichao Yu. What English na...             0  
Removing the following lines: 
                                           question1  \
test_id                                                
379205      How I can learn android app development?   
817520   How real can learn android app development?   
943911                          How app dev

## Tokenization 

See (https://paperswithcode.com/method/wordpiece)

In [54]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [20]:
print("Token id for [CLS]: " + str(tokenizer.cls_token_id))
print("Token id for [SEP]: " + str(tokenizer.sep_token_id))
print("Token id for [PAD]: " + str(tokenizer.pad_token_id))
print("Token id for [UNK]: " + str(tokenizer.unk_token_id))
print("Token id for [MASK]: " + str(tokenizer.mask_token_id))

print("Original sentense: " + train.loc[0, 'question1'])
print("Encoded sentense: ")
enc = tokenizer.encode(train.loc[0, 'question1'])
print(enc)
print("Decoded sentense: ")
dec = tokenizer.decode(enc)
print(dec)

# Check len of tokenized training sentences:
list_len = []
all_s = train['question1'].tolist() + train['question2'].tolist()
for s in all_s:
    tks = tokenizer.encode(s)
    list_len.append(len(tks))

max_len = max(list_len)

print(f"La taille maximale de tokens est {max_len} (avec les [CLS] et [SEP])")
lw_64 = round((sum([l <= 64 for l in list_len])/len(list_len)) * 100, 2) 
print(f"Il y a {lw_64}% des phrases tokenised qui sont <= 64. C'est suffisant, on supprimera celle plus grande du dataset")

# Check for the sentences pairs
list_len2 = []
for i in range(train.shape[0]):
    tks = tokenizer.encode(train['question1'][i], train['question2'][i])
    list_len2.append(len(tks))

max_len2 = max(list_len2)
print(f"La taille maximale de tokens avec paires mergée est {max_len2} (avec les [CLS] et [SEP])")
lw_64 = round((sum([l <= 64 for l in list_len2])/len(list_len2)) * 100, 2)
lw_128 = round((sum([l <= 128 for l in list_len2])/len(list_len2)) * 100, 2)
print(f"Il y a {lw_64}% des paires de phrases mergées tokenised qui sont <= 64. C'est suffisant, on supprimera celle plus grande du dataset")
print(f"Il y a {lw_128}% des paires de  phrases mergées tokenised qui sont <= 64. C'est suffisant, on supprimera celle plus grande du dataset")


Token id for [CLS]: 101
Token id for [SEP]: 102
Token id for [PAD]: 0
Token id for [UNK]: 100
Token id for [MASK]: 103
Original sentense: What is the step by step guide to invest in share market in india?
Encoded sentense: 
[101, 2054, 2003, 1996, 3357, 2011, 3357, 5009, 2000, 15697, 1999, 3745, 3006, 1999, 2634, 1029, 102]
Decoded sentense: 
[CLS] what is the step by step guide to invest in share market in india? [SEP]
La taille maximale de tokens est 286 (avec les [CLS] et [SEP])
Il y a 99.88% des phrases tokenised qui sont <= 64. C'est suffisant, on supprimera celle plus grande du dataset
La taille maximale de tokens avec paires mergée est 330 (avec les [CLS] et [SEP])
Il y a 97.91% des paires de phrases mergées tokenised qui sont <= 64. C'est suffisant, on supprimera celle plus grande du dataset
Il y a 99.97% des paires de  phrases mergées tokenised qui sont <= 64. C'est suffisant, on supprimera celle plus grande du dataset


# Data loading

See https://pytorch.org/tutorials/beginner/basics/data_tutorial.html for documentation about the Dataset and Dataloader creation.

In [55]:
class SiameseNetWorkSentenceDataset(Dataset):
    """
    SiameseNetWorkSentenceDataset create a Dataset
    - data (pd.DataFrame): the data dataframe with column 'question1' and 'question2' along with the label 'is_duplicate'
    - tokenizer: the BERT tokenizer, such as: BertTokenizer.from_pretrained('bert-base-uncased')
    - max_length: the maximal length of tokens input vector (default 64) Shorter vector arre padded to max_length with [PAD token] (id: 0) and longer are truncated. 
    The size includes the start [CLS] and end [SEP] tokens.
    """

    def __init__(self, data, tokenizer, max_length):
        self.data = data
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return self.data.shape[0]
    
    def __getitem__(self, index):
        
        def squeeze_tensors(tks):
            """Take a tensor and remove unnecessary dimension. When using tokenizer with return_tensors = 'pt', the returned tensor is by default 2 dimensions, has it could handle a list of sentence as inputs.
            However, as we only sent one sentence at a time to the tokenizer to create the Dataset, it result in an additional dimension that will be useless after pooling results by batches in the DataLoader

            Args:
                tks ([type]): [description]
            """
            tks.data["input_ids"] = torch.squeeze(tks.data["input_ids"])
            tks.data["token_type_ids"] = torch.squeeze(tks.data["token_type_ids"])
            tks.data["attention_mask"] = torch.squeeze(tks.data["attention_mask"])

        s1 = self.data.loc[index, 'question1']
        s2 = self.data.loc[index, 'question2']
        label = torch.tensor(self.data.loc[index, 'is_duplicate'])

        tokens1 = self.tokenizer(text = s1, max_length = self.max_length, padding = 'max_length', truncation = True, return_tensors = 'pt')
        squeeze_tensors(tokens1)
        tokens2 = self.tokenizer(text = s2, max_length = self.max_length, padding = 'max_length', truncation = True, return_tensors = 'pt')
        squeeze_tensors(tokens2)

        return tokens1, tokens2, label


class BERTSentencesClassificationDataset(Dataset):
    
    def __init__(self, data, tokenizer, max_length):
        self.data = data
        self.tokenizer = tokenizer
        self.max_length = max_length
    
    def __len__(self):
        return self.data.shape[0]
    
    def __getitem__(self, index):

        def squeeze_tensors(tks):
            """Take a tensor and remove unnecessary dimension. When using tokenizer with return_tensors = 'pt', the returned tensor is by default 2 dimensions, has it could handle a list of sentence as inputs.
            However, as we only sent one sentence at a time to the tokenizer to create the Dataset, it result in an additional dimension that will be useless after pooling results by batches in the DataLoader

            Args:
                tks ([type]): [description]
            """
            tks.data["input_ids"] = torch.squeeze(tks.data["input_ids"])
            tks.data["token_type_ids"] = torch.squeeze(tks.data["token_type_ids"])
            tks.data["attention_mask"] = torch.squeeze(tks.data["attention_mask"])
        
        s1 = self.data.loc[index, 'question1']
        s2 = self.data.loc[index, 'question2']

        t = self.tokenizer(s1, s2, max_length = self.max_length, padding = 'max_length', truncation = True, return_tensors = 'pt')
        squeeze_tensors(t)

        label = torch.tensor(self.data.loc[index, 'is_duplicate'])

        return t, label


In [56]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
train_dataset = SiameseNetWorkSentenceDataset(data = train, tokenizer = tokenizer, max_length = 64)
print(train_dataset[10])
print(len(train_dataset))

train_dataset_2 = BERTSentencesClassificationDataset(data = train, tokenizer = tokenizer, max_length = 64)
print(train_dataset_2[10])
print(len(train_dataset_2))

({'input_ids': tensor([  101,  4118,  2000,  2424,  8745,  1997, 29199,  2478, 10424,  2229,
        11877, 12170, 18098,  2964,  1029,   102,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0]), 'token_type_ids': tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), 'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])}, {'inpu

In [57]:
dataloader = DataLoader(train_dataset, batch_size = 8, shuffle = True, num_workers = 0)
dataloader_2 = DataLoader(train_dataset_2, batch_size = 8, shuffle = True, num_workers = 0)

# Dataset details

Each input in the training/test dataset is composed of question1, question2, and label
The Dataset in shuffled / divided in batchs of size *batch_size* in the DataLoader.

## SiameseNetWorkSentenceDataset

For the siamese network BERT model, a batch is a tuple of 3 elements: (*batch_question_1*, *batch_question_2*, *label*).

For the *batch_size* pair of questions in the created batch: 
- *batch_question_1*: contains the BERT-tokenizer's output of each question1
- *batch_question_2*: contains the BERT-tokenizer's output of each question2
- *label* indicates if the two questions are duplicated 

*batch_question_1* and *batch_question_2* are similar and contains a dictionary with 3 entries:

- "input_ids": torch.FloatTensor of shape (*batch_size*, *sequence_length*) which contains the indices of the question tokens in the vocabulary, with [CLS] and [SEP] at the beginning and end of the sentence and the [PAD] token for the remaining padding (up to *max_length*).

- "token_type_ids": torch.FloatTensor of shape (*batch_size*, *sequence_length*) which are the segment token indices to indicate first and second portions of the inputs. Indices are selected in [0, 1]. Always 0 here as the two questions are treated separately in the siamese network.

- attention_mask torch.FloatTensor of shape (*batch_size*, *sequence_length* which contains the mask to avoid performing attention on padding token indices. Mask values selected in [0, 1]

When calling the model SiameseBERTNet, *batch_question_1* is fed in the 'left' part of the siamese and *batch_question_2* is fed in the right part of the siamese. In each siamese, we compute for each question the average of the last hidden layer along each 'real' token of the question, so that we don't average with [PAD], [CLS] and [SEP] tokens.
At the end, for each *batch_size* pair of questions sent to the siamese Network, it returns a tuple with the two set of averaged vectors for question1 and question2, noted (OUT1, OUT2), each of shape (*batch_size*, dmodel). dmodel of BERT is 768. The ième line in OUT1 correspond to the averaged output hidden layer for the ième question1 in *batch_question_1*.

In the loss function, we want to minimze the distance between 2 averaged vector OUT1[i, ] and OUT2[i, ] if they are duplicated and maximize the distance if they are not duplicated

## BERTSentencesClassification

For BERTSentence classification we choose to directly use the [CLS] token as a predictor like it is done for the NextSentencePrediction task.
Each batch is a tuple of 2 elements (*batch_pair_of_questions*, *label*).

*batch_pair_of_question* is a dictionary like *batch_question_1* or *batch_question_1* with each elements (input_ids, token_type_ids, attention_mask) of shape (*batch_size*, *sequence_length*), except that here the two question have been concatened into one sequence, such as : [CLS] [... TOKEN Q1 ...] [SEP] [... TOKEN Q2 ...] [SEP]. In this case *token_type_ids* matrix is important as we have to distinguish the both sentences in the embedding. The advantage of this approach is that the attention of each tokens is computed over all the tokens of the sequence, included those of the other question, while the siamese network compute the averaged output token independenty for each question.

After being fed into the BERT model, the outputed vector corresponding to the [CLS] token, namely pooler_output, in sent into a dropout, linear layer with two outputed dimension and finally a softmax. The goal is to predict is the two concatenated questions are duplicated or not. 



# Model

In [58]:
class SiameseBERTNet(nn.Module):

    def __init__(self):
        super(SiameseBERTNet, self).__init__()

        self.bert = BertModel.from_pretrained('bert-base-uncased')
        self.HS = self.bert.config.hidden_size

    def forward_siamese(self, input, noCLSpooling = True, noSEPpooling = True):
        """From tokenised input sentence, compute BERT 

        Args:
            input (dict): output dict from the tokenizer with input_ids, token_type_ids and attention_mask

        Returns:
            avg (tensor): Mean of the last hidden layer vectors for real tokens (attention_mask: 1) in the input.
        """
        # Get input_ids and attention mask
        input_ids, token_type_ids, attention_mask = input.values()

        # Apply BERT and extract last_hidden_state
        out = self.bert(input_ids = input_ids, attention_mask = attention_mask)
        last_hidden_state = out.last_hidden_state

        # Apply mean pooling on real tokens
        # Make a copy is some changes (CLS or SEP) need to be applied
        pooling_mask = attention_mask.detach().clone()

        # If the CLS output vector should not participate in average pooling
        if noCLSpooling:
            pooling_mask[:, 0] = 0
        
        # If the SEP output vector should not participate in average pooling
        if noSEPpooling:
            pooling_mask = torch.where(input_ids == 102, 0, pooling_mask)

        # Get mask at the same dimension as last_hidden_state
        expanded_pooling_mask = pooling_mask.unsqueeze(-1)
        expanded_pooling_mask = expanded_pooling_mask.expand(-1, -1, self.HS)

        # Element wise mul between last_hidden_state and mask to then only consider real tokens in the sum
        prod = torch.mul(last_hidden_state, expanded_pooling_mask)

        # Sum all token vectors
        sum_by_tks = torch.sum(prod, dim = 1)

        # Get normalisation factor to compute mean
        norm = torch.sum(pooling_mask, dim = -1).unsqueeze(-1)

        # Comptue average
        avg = torch.div(sum_by_tks, norm)

        return avg


    def forward(self, question1, question2):
        out1 = self.forward_siamese(question1)
        out2 = self.forward_siamese(question2)

        return out1, out2

class BERTSentencesClassification(nn.Module):

    def __init__(self):
        super(BERTSentencesClassification, self).__init__()

        self.bert = BertModel.from_pretrained('bert-base-uncased')
        self.HS = self.bert.config.hidden_size
        self.out = 2
        self.classifier = nn.Sequential(
            nn.Dropout(p = 0.2),
            nn.Linear(in_features = self.HS, out_features = self.out, bias = True),
            nn.Softmax(dim = 1)
        )


    def forward(self, sentensepair):

        # Get input_ids, token_type_ids (as we have sentense pairs) and attention mask
        out = self.bert(**sentensepair)
        cls_token = out.pooler_output
        classification = self.classifier(cls_token)

        return classification, cls_token


also see https://skimai.com/fine-tuning-bert-for-sentiment-analysis/ for tips

In [59]:
model = SiameseBERTNet()
model2 = BERTSentencesClassification()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.pred

For details about the last 'real' layer BertPooler see:  https://github.com/huggingface/transformers/issues/782 and https://github.com/google-research/bert/issues/43. Il s'agit d'une tranformation appliquée uniquement au token [CLS]. L'article ne détaille pas tout mais en réalité il y a donc un layer linéaire de transformation appliquée sur le token CLS avant de l'envoyé dans un linear layer de classification/softmax.

In [60]:
s1 = next(iter(dataloader))
r = model(s1[0], s1[1])
s2 = next(iter(dataloader_2))
r_2 = model2(s2[0])

# Test BERT embedding without fine-tuning

In [61]:
# Extract a set of sample sentenses from test set:

positives_s = test.head(n = 10)
negatives_s = test.tail(n = 10)

all_s = positives_s.append(negatives_s, ignore_index = True)
all_s.to_csv("data/sample.csv", index = False)
d = SiameseNetWorkSentenceDataset(data = all_s, tokenizer = tokenizer, max_length = 64)
l = DataLoader(d, batch_size = 20, shuffle = False, num_workers = 0)
sample = next(iter(l))
res = model(sample[0], sample[1])
res = torch.cat([res[0], res[1]], axis = 0)

# Compute PCA (1)
U, S, V = torch.pca_lowrank(res, niter = 50)
proj = torch.matmul(res, V[:, :2])

plot_data = pd.DataFrame(proj.detach().numpy())
allQ = (all_s["question1"].tolist() + all_s["question2"].tolist())
plot_data["question"] = allQ
plot_data = plot_data.rename({0: 'X', 1: 'Y'}, axis=1)

fig = px.scatter(plot_data, x = "X", y = "Y", hover_data=['question'])
fig.show()

# Compute distance matrix :
D = torch.cdist(res.double(), res.double(), p = 2)
D = D.detach().numpy().round(5)
# Get the closest sentence for positives examples : 
for i in range(10):
    closest = np.argsort(D[i])[1]
    print(allQ[i]  + " -> closest: " + allQ[closest] + ". Real label is: " + allQ[i + 20])

df = pd.DataFrame(D)

df.to_csv("data/NormEucDistanceMatrix_1.csv", index = False, header = False)


# For the second model:
d_2 = BERTSentencesClassificationDataset(data = all_s, tokenizer = tokenizer, max_length = 64)
l_2 = DataLoader(d_2, batch_size = 20, shuffle = False, num_workers = 0)
sample_2 = next(iter(l_2))

res_2 = model2(sample_2[0])

# Compute PCA (2)
U2, S2, V2 = torch.pca_lowrank(res_2[1], niter = 50)
proj_2 = torch.matmul(res_2[1], V2[:, :2])

plot_data_2 = pd.DataFrame(proj_2.detach().numpy())
plot_data_2["question"] = "Q1: " + all_s["question1"] + "\nQ2: " + all_s["question2"]
plot_data_2["label"] = all_s["is_duplicate"]
plot_data_2 = plot_data_2.rename({0: 'X', 1: 'Y'}, axis=1)
plot_data_2 = plot_data_2.astype(dtype = {'label': 'str'}, copy = True)

fig_2 = px.scatter(plot_data_2, x = "X", y = "Y", color = "label", hover_data=['question'])
fig_2.show()

How does the Surface Pro himself 4 compare with iPad Pro? -> closest: Where is fingerprint involve of LEO Privacy Guard? How does it work?. Real label is: Why did Microsoft choose core m3 and not core i3 home Surface Pro 4?
Should I have a hair transplant at age 24? How much would it cost? -> closest: How much cost does hair transplant require?. Real label is: How much cost does hair transplant require?
What but is the best way to send money from China to the US? -> closest: What you send money to China?. Real label is: What you send money to China?
Which food not emulsifiers? -> closest: What foods fibre?. Real label is: What foods fibre?
How "aberystwyth" start reading? -> closest: What are the best and worst things examination public transit in Visakhapatnam, Andhra Pradesh, India? How could it be improved?. Real label is: How their can I start reading?
How are the two wheeler insurance from Bharti Axa insurance? -> closest: By scrapping the 500 and 1000 rupee notes, how is RBI plan